## Deliverable 2. Create a Customer Travel Destinations Map.

In [15]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [16]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,San Patricio,US,28.0170,-97.5169,77.09,42,34,7.09,scattered clouds
1,1,Hobart,AU,-42.8794,147.3294,59.27,88,100,1.14,overcast clouds
2,2,North Bend,US,43.4065,-124.2243,53.51,80,75,11.50,broken clouds
3,3,Katsuura,JP,35.1333,140.3000,54.19,59,0,32.50,clear sky
4,4,Dingle,PH,10.9995,122.6711,75.49,91,100,7.96,overcast clouds


In [17]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 75


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [19]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                131
City                   131
Country                129
Lat                    131
Lng                    131
Max Temp               131
Humidity               131
Cloudiness             131
Wind Speed             131
Current Description    131
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Hermanus,ZA,65.89,overcast clouds,-34.4187,19.2345,
8,Port Elizabeth,ZA,61.11,scattered clouds,-33.9180,25.5701,
12,Cape Town,ZA,66.11,few clouds,-33.9258,18.4232,
14,Hualmay,PE,70.23,overcast clouds,-11.0964,-77.6139,
19,Lagoa,PT,66.18,overcast clouds,39.0500,-27.9833,
30,Ribeira Grande,PT,68.40,scattered clouds,38.5167,-28.7000,
34,Constitucion,CL,64.69,clear sky,-35.3333,-72.4167,
35,Sirsaganj,IN,64.53,clear sky,27.0500,78.7000,
36,Port Alfred,ZA,62.37,overcast clouds,-33.5906,26.8910,
45,Lebu,CL,61.43,clear sky,-37.6167,-73.6500,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [23]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")

hotel_df.replace("", nan_value, inplace=True)

hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
clean_hotel_df = hotel_df

In [24]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))